In [1]:
import sys, os

# For .py
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
# For .ipynb
sys.path.append(os.path.dirname(os.getcwd()))

import json
import random
from util import Image_Loader
from util import find_random_question
from util import random_word
from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
import re
import ast

In [2]:
default_model="gpt-4o-mini"
verbose=True

total_cost = 0
keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_2'
# random_question = find_random_question(keyword, problem_type, detail_type=1, count=2)
# random_question

### Create a model for generating a problem
models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.5, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)

### get random word
main_topic = random_word()
if verbose:
    print(main_topic)

## Generate a new word based on the random word
problem_generate_model.request_models_responses(
    [
        SystemMessage(content="주제어와 관련된 일상적인 명사 1개를 제시하라. no intro. no outro."),
        HumanMessage(content="주제어: " + main_topic + " 명사: "),
    ]
)
word_responses = problem_generate_model.get_model_responses()
main_topic_generated = word_responses[0][1]
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))


입술
gpt-4o-mini-2024-07-18 | 
립밤

OpenAI Tokens Used: 49
    Prompt Tokens: 47
    Completion Tokens: 2
Successful Requests: 1
Total Cost (USD): $0.0001
Total Cost (KRW): ₩0.135
--------------------------------------------------------------------------------------------------


- 문제 생성

In [3]:
example_question = {'제목': '대출 금리 하락세, 부동산 시장 기지개',
                    '정답': ['대출 금리가 떨어지면서 부동산 시장이 살아나기 시작했다.'],
                    '오답': ['부동산 시장에 대한 규제가 대출 금리 하락에 영향을 미쳤다.', '대출 금리가 하락했지만 부동산 시장의 거래는 줄어들고 있다.', '부동산 시장을 활성화하려고 대출 금리 안정화 대책이 논의되고 있다.']
    }
human_prompt = f"""주제어: {main_topic_generated}."""
display(human_prompt)
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content=f"너는 신문 기사의 제목을 만드는 봇이다. 기사 제목을 가장 잘 설명하는 문장 1개, 아닌 문장 3개를 만들어라. 예시: {example_question}"
        ),
        HumanMessage(content=human_prompt),
    ]
)

response = problem_generate_model.get_model_responses()

total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', response[0][2]).group(1))

response = response[0][1]
response = ast.literal_eval(response)
display(response)

# target_phrase = None
# similar_phrase = None
# for phrase in phrase_list:
#     if phrase in sentence_response:
#         target_phrase = phrase
#     else:
#         similar_phrase = phrase
# assert target_phrase is not None, "Target phrase not found in the sentence"
# assert similar_phrase is not None, "Similar phrase not found in the sentence"
# print(f'Target Phrase: {target_phrase}\nSimilar Phrase: {similar_phrase}')

'주제어: 립밤.'

gpt-4o-mini-2024-07-18 | 
{'제목': '립밤, 겨울철 입술 보호의 필수 아이템', '정답': ['겨울철 건조한 날씨에 립밤이 필수적으로 사용되고 있다.'], '오답': ['립밤의 가격이 최근에 상승했다.', '립밤의 성분에 대한 논란이 일고 있다.', '많은 사람들이 립밤 대신 다른 보습 제품을 선호하고 있다.']}

OpenAI Tokens Used: 270
    Prompt Tokens: 175
    Completion Tokens: 95
Successful Requests: 1
Total Cost (USD): $0.00054
Total Cost (KRW): ₩0.729
--------------------------------------------------------------------------------------------------


{'제목': '립밤, 겨울철 입술 보호의 필수 아이템',
 '정답': ['겨울철 건조한 날씨에 립밤이 필수적으로 사용되고 있다.'],
 '오답': ['립밤의 가격이 최근에 상승했다.',
  '립밤의 성분에 대한 논란이 일고 있다.',
  '많은 사람들이 립밤 대신 다른 보습 제품을 선호하고 있다.']}

In [4]:
result = {
    "example": response['제목'],
    "selector": response['정답'] + response['오답'],
    "answer": response['정답'][0],
    "eval_answer": response['정답'][0],
    "total_cost": total_cost,
}
result

{'example': '립밤, 겨울철 입술 보호의 필수 아이템',
 'selector': ['겨울철 건조한 날씨에 립밤이 필수적으로 사용되고 있다.',
  '립밤의 가격이 최근에 상승했다.',
  '립밤의 성분에 대한 논란이 일고 있다.',
  '많은 사람들이 립밤 대신 다른 보습 제품을 선호하고 있다.'],
 'answer': ['겨울철 건조한 날씨에 립밤이 필수적으로 사용되고 있다.'],
 'eval_answer': ['겨울철 건조한 날씨에 립밤이 필수적으로 사용되고 있다.'],
 'total_cost': 0.864}